In [2]:
import os
import glob
import math
import random
import shutil

from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn.functional as F

# Reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
class PairedImageDataset(Dataset):
    def __init__(self, lr_root, hr_root, training=True, patch_size=128):
        super().__init__()
        self.lr_root = lr_root
        self.hr_root = hr_root
        self.training = training
        self.patch_size = patch_size

        self.lr_paths = sorted(
            [p for p in glob.glob(os.path.join(lr_root, "*")) if os.path.isfile(p)]
        )
        self.hr_paths = sorted(
            [p for p in glob.glob(os.path.join(hr_root, "*")) if os.path.isfile(p)]
        )

        assert len(self.lr_paths) == len(self.hr_paths) and len(self.lr_paths) > 0, \
            "Mismatch or empty LR/HR folders"

        self.to_tensor = T.ToTensor()

    def __len__(self):
        return len(self.lr_paths)

    def _random_augment(self, lr_img, hr_img):
        # flips
        if random.random() < 0.5:
            lr_img = lr_img.transpose(Image.FLIP_LEFT_RIGHT)
            hr_img = hr_img.transpose(Image.FLIP_LEFT_RIGHT)
        if random.random() < 0.5:
            lr_img = lr_img.transpose(Image.FLIP_TOP_BOTTOM)
            hr_img = hr_img.transpose(Image.FLIP_TOP_BOTTOM)
        # rotations
        k = random.randint(0, 3)
        if k:
            lr_img = lr_img.rotate(90 * k, expand=False)
            hr_img = hr_img.rotate(90 * k, expand=False)
        return lr_img, hr_img

    def _random_crop_pair(self, lr_img, hr_img):
        w, h = lr_img.size
        ps = self.patch_size

        if w <= ps or h <= ps:
            return lr_img, hr_img

        x = random.randint(0, w - ps)
        y = random.randint(0, h - ps)

        lr_patch = lr_img.crop((x, y, x + ps, y + ps))
        hr_patch = hr_img.crop((x, y, x + ps, y + ps))
        return lr_patch, hr_patch

    def __getitem__(self, idx):
        lr_path = self.lr_paths[idx]
        hr_path = self.hr_paths[idx]

        lr = Image.open(lr_path).convert("RGB")
        hr = Image.open(hr_path).convert("RGB")

        target_size = (256, 256)
        if lr.size != target_size:
            lr = lr.resize(target_size, Image.BICUBIC)
        if hr.size != target_size:
            hr = hr.resize(target_size, Image.BICUBIC)

        if self.training:
            lr, hr = self._random_augment(lr, hr)
            lr, hr = self._random_crop_pair(lr, hr)

        lr = self.to_tensor(lr)  # [0,1]
        hr = self.to_tensor(hr)

        return lr, hr


In [4]:
train_lr_root = "LR_train/train"
train_hr_root = "HR_train/train"
val_lr_root   = "LR_val/val"
val_hr_root   = "HR_val/val"

patch_size = 128

train_dataset = PairedImageDataset(train_lr_root, train_hr_root,
                                   training=True, patch_size=patch_size)
val_dataset   = PairedImageDataset(val_lr_root,   val_hr_root,
                                   training=False, patch_size=patch_size)

batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=1,
                          shuffle=False, num_workers=2, pin_memory=True)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 258
Val samples: 100


In [5]:
def calc_psnr_rgb(pred, target):
    pred = torch.clamp(pred, 0.0, 1.0)
    target = torch.clamp(target, 0.0, 1.0)
    mse = torch.mean((pred - target) ** 2).item()
    if mse == 0:
        return 99.0
    return 10 * math.log10(1.0 / mse)

def rgb_to_y(img):
    r, g, b = img[:, 0:1], img[:, 1:2], img[:, 2:3]
    return 0.299 * r + 0.587 * g + 0.114 * b

def calc_psnr_y(pred, target):
    pred = torch.clamp(pred, 0.0, 1.0)
    target = torch.clamp(target, 0.0, 1.0)
    py = rgb_to_y(pred)
    ty = rgb_to_y(target)
    mse = torch.mean((py - ty) ** 2).item()
    if mse == 0:
        return 99.0
    return 10 * math.log10(1.0 / mse)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


In [6]:
class DWConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, dilation=1):
        super().__init__()
        self.dw = nn.Conv2d(
            in_channels, in_channels,
            kernel_size=kernel_size,
            padding=padding,
            dilation=dilation,
            groups=in_channels
        )
        self.pw = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)
        return x


In [7]:
class SMFA_Nano(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.channels = channels
        self.expand = nn.Conv2d(channels, 2 * channels, kernel_size=1)

        # EASA branch
        self.pool = nn.AvgPool2d(kernel_size=4, stride=4)  # simple downsample
        self.easa_dw = nn.Conv2d(channels, channels, kernel_size=3, padding=1, groups=channels)
        self.easa_fuse = nn.Conv2d(channels, channels, kernel_size=1)

        # LDE branch
        self.lde_conv1 = DWConv(channels, channels, kernel_size=3, padding=1)
        self.lde_conv2 = DWConv(channels, channels, kernel_size=3, padding=1)

        # fuse
        self.fuse_conv = nn.Conv2d(2 * channels, channels, kernel_size=1)

        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        B, C, H, W = x.shape
        x_exp = self.expand(x)
        x_easa, x_lde = torch.split(x_exp, self.channels, dim=1)

        # --- EASA: non-local branch ---
        low = self.pool(x_easa)                          # [B,C,H/4,W/4]
        low = self.easa_dw(low)
        # variance modulation (approx)
        mean = x_easa.mean(dim=[2, 3], keepdim=True)
        var = ((x_easa - mean) ** 2).mean(dim=[2, 3], keepdim=True)
        var = self.easa_fuse(var)                        # channel modulation
        var = torch.sigmoid(var)
        # upsample back & gate
        nonlocal_feat = F.interpolate(low, size=(H, W), mode="nearest")
        nonlocal_feat = nonlocal_feat * var

        # --- LDE: local detail branch ---
        local = self.act(self.lde_conv1(x_lde))
        local = self.act(self.lde_conv2(local))

        # fuse
        out = torch.cat([nonlocal_feat, local], dim=1)
        out = self.fuse_conv(out)
        return out + x


class PCFN_Nano(nn.Module):
    """Partial Conv Feed-Forward Network (tiny)."""
    def __init__(self, channels, expansion=2):
        super().__init__()
        hidden = channels * expansion
        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1)
        self.dw = nn.Conv2d(hidden, hidden, kernel_size=3, padding=1, groups=hidden)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.act(x)
        x = self.dw(x)
        x = self.act(x)
        x = self.fc2(x)
        return residual + x


class VPEGBlock_Nano(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.smfa = SMFA_Nano(channels)
        self.pcf = PCFN_Nano(channels, expansion=2)

    def forward(self, x):
        x = self.smfa(x)
        x = self.pcf(x)
        return x


class VPEG_NanoRestorer(nn.Module):
    """
    VPEG C–inspired nano model for 1x restoration (256x256 -> 256x256).
    """
    def __init__(self,
                 in_channels=3,
                 out_channels=3,
                 channels=32,
                 num_blocks=4):
        super().__init__()
        self.shallow = nn.Conv2d(in_channels, channels, kernel_size=3, padding=1)

        self.blocks = nn.ModuleList([
            VPEGBlock_Nano(channels) for _ in range(num_blocks)
        ])

        self.conv_mid = DWConv(channels, channels, kernel_size=3, padding=1)
        self.conv_out = nn.Conv2d(channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        inp = x
        feat = self.shallow(x)

        x_mid = feat
        for blk in self.blocks:
            x_mid = blk(x_mid)

        x_mid = self.conv_mid(x_mid)
        feat = feat + x_mid
        out = self.conv_out(feat)
        return inp + out


In [8]:
class PLKB_Nano(nn.Module):
    """
    Partial Large Kernel Block:
    - split channels in half
    - large-kernel depthwise conv on one half
    - concat with bypass + 1x1 conv mix
    """
    def __init__(self, channels, k=7, dilation=2):
        super().__init__()
        self.channels = channels
        self.c_half = channels // 2
        self.c_rem = channels - self.c_half

        pad = dilation * (k // 2)
        self.dw_large = nn.Conv2d(
            self.c_half, self.c_half,
            kernel_size=k,
            padding=pad,
            dilation=dilation,
            groups=self.c_half
        )
        self.mix = nn.Conv2d(channels, channels, kernel_size=1)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x1, x2 = torch.split(x, [self.c_half, self.c_rem], dim=1)
        x1 = self.act(self.dw_large(x1))
        x_cat = torch.cat([x1, x2], dim=1)
        out = self.mix(x_cat)
        return out


class SpatialFreqAttention_Nano(nn.Module):
    """
    Frequency-inspired attention but implemented with large depthwise convs.
    """
    def __init__(self, channels):
        super().__init__()
        # build a 1-channel attention map
        self.dw1 = nn.Conv2d(1, 1, kernel_size=5, padding=2, groups=1)
        self.dw2 = nn.Conv2d(1, 1, kernel_size=7, padding=3, groups=1)
        self.fuse = nn.Conv2d(2, 1, kernel_size=1)
        self.act = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        # x: B,C,H,W
        s = x.mean(dim=1, keepdim=True)   # pseudo "frequency summary"
        a1 = self.act(self.dw1(s))
        a2 = self.act(self.dw2(s))
        a = torch.cat([a1, a2], dim=1)
        a = self.fuse(a)
        a = torch.sigmoid(a)
        return x * a  # pixel attention


class ZHEstarBlock_Nano(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.plkb = PLKB_Nano(channels, k=7, dilation=2)
        self.att  = SpatialFreqAttention_Nano(channels)
        self.conv_out = DWConv(channels, channels, kernel_size=3, padding=1)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        x = self.plkb(x)
        x = self.act(x)
        x = self.att(x)
        x = self.conv_out(x)
        return residual + x


class ZHEstar_NanoRestorer(nn.Module):
    """
    ZHEstar-inspired nano model for 1x restoration.
    """
    def __init__(self,
                 in_channels=3,
                 out_channels=3,
                 channels=32,
                 num_blocks=4):
        super().__init__()
        self.shallow = nn.Conv2d(in_channels, channels, kernel_size=3, padding=1)
        self.blocks = nn.ModuleList([
            ZHEstarBlock_Nano(channels) for _ in range(num_blocks)
        ])
        self.conv_mid = DWConv(channels, channels, kernel_size=3, padding=1)
        self.conv_out = nn.Conv2d(channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        inp = x
        feat = self.shallow(x)

        x_mid = feat
        for blk in self.blocks:
            x_mid = blk(x_mid)

        x_mid = self.conv_mid(x_mid)
        feat = feat + x_mid
        out = self.conv_out(feat)
        return inp + out


In [9]:
vpeg_model = VPEG_NanoRestorer(in_channels=3, out_channels=3,
                               channels=32, num_blocks=4).to(device)
zhestar_model = ZHEstar_NanoRestorer(in_channels=3, out_channels=3,
                                     channels=32, num_blocks=4).to(device)

print("VPEG_Nano params:", count_parameters(vpeg_model),
      f"({count_parameters(vpeg_model)/1e6:.6f} M)")
print("ZHEstar_Nano params:", count_parameters(zhestar_model),
      f"({count_parameters(zhestar_model)/1e6:.6f} M)")


VPEG_Nano params: 55747 (0.055747 M)
ZHEstar_Nano params: 16383 (0.016383 M)


In [17]:
model_choice = "zhestar"  # or "zhestar"

channels   = 32
num_blocks = 4

if model_choice == "vpeg":
    model = VPEG_NanoRestorer(in_channels=3, out_channels=3,
                              channels=channels, num_blocks=num_blocks).to(device)
else:
    model = ZHEstar_NanoRestorer(in_channels=3, out_channels=3,
                                 channels=channels, num_blocks=num_blocks).to(device)

print(model_choice, "params:", count_parameters(model),
      f"({count_parameters(model)/1e6:.6f} M)")


zhestar params: 16383 (0.016383 M)


In [18]:
criterion = nn.MSELoss()

num_epochs = 1000       # can reduce to 400 if it feels too long
initial_lr = 5e-4

optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=num_epochs, eta_min=1e-6
)

ckpt_dir = f"checkpoints_{model_choice}_nano"
os.makedirs(ckpt_dir, exist_ok=True)

best_val_psnr_rgb = 0.0
best_epoch        = 0


In [19]:
for epoch in range(1, num_epochs + 1):
    # ---------- TRAIN ----------
    model.train()
    running_loss = 0.0

    for lr_img, hr_img in train_loader:
        lr_img = lr_img.to(device, non_blocking=True)
        hr_img = hr_img.to(device, non_blocking=True)

        optimizer.zero_grad()
        sr = model(lr_img)
        loss = criterion(sr, hr_img)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * lr_img.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # ---------- VAL ----------
    model.eval()
    val_psnr_rgb_sum = 0.0
    val_psnr_y_sum   = 0.0

    with torch.no_grad():
        for lr_img, hr_img in val_loader:
            lr_img = lr_img.to(device, non_blocking=True)
            hr_img = hr_img.to(device, non_blocking=True)

            sr = model(lr_img)
            val_psnr_rgb_sum += calc_psnr_rgb(sr, hr_img)
            val_psnr_y_sum   += calc_psnr_y(sr, hr_img)

    val_psnr_rgb = val_psnr_rgb_sum / len(val_loader)
    val_psnr_y   = val_psnr_y_sum / len(val_loader)

    scheduler.step()

    print(f"[{model_choice.upper()}] Epoch [{epoch}/{num_epochs}]  "
          f"Train MSE: {train_loss:.6f}  "
          f"Val PSNR(RGB): {val_psnr_rgb:.4f} dB  "
          f"Val PSNR(Y): {val_psnr_y:.4f} dB")

    # save last
    torch.save(
        {
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "val_psnr": val_psnr_rgb,
        },
        os.path.join(ckpt_dir, "last_model.pt"),
    )

    # save best
    if val_psnr_rgb > best_val_psnr_rgb:
        best_val_psnr_rgb = val_psnr_rgb
        best_epoch        = epoch
        torch.save(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "val_psnr": val_psnr_rgb,
            },
            os.path.join(ckpt_dir, "best_model.pt"),
        )
        print(f"  -> New BEST RGB PSNR: {best_val_psnr_rgb:.4f} dB at epoch {best_epoch}")


[ZHESTAR] Epoch [1/1000]  Train MSE: 0.019816  Val PSNR(RGB): 18.7667 dB  Val PSNR(Y): 20.2665 dB
  -> New BEST RGB PSNR: 18.7667 dB at epoch 1
[ZHESTAR] Epoch [2/1000]  Train MSE: 0.014190  Val PSNR(RGB): 19.2079 dB  Val PSNR(Y): 20.6018 dB
  -> New BEST RGB PSNR: 19.2079 dB at epoch 2
[ZHESTAR] Epoch [3/1000]  Train MSE: 0.012632  Val PSNR(RGB): 19.5226 dB  Val PSNR(Y): 20.8228 dB
  -> New BEST RGB PSNR: 19.5226 dB at epoch 3
[ZHESTAR] Epoch [4/1000]  Train MSE: 0.011322  Val PSNR(RGB): 19.8153 dB  Val PSNR(Y): 21.0073 dB
  -> New BEST RGB PSNR: 19.8153 dB at epoch 4
[ZHESTAR] Epoch [5/1000]  Train MSE: 0.011011  Val PSNR(RGB): 20.0796 dB  Val PSNR(Y): 21.1781 dB
  -> New BEST RGB PSNR: 20.0796 dB at epoch 5
[ZHESTAR] Epoch [6/1000]  Train MSE: 0.009963  Val PSNR(RGB): 20.3467 dB  Val PSNR(Y): 21.3262 dB
  -> New BEST RGB PSNR: 20.3467 dB at epoch 6
[ZHESTAR] Epoch [7/1000]  Train MSE: 0.009692  Val PSNR(RGB): 20.5771 dB  Val PSNR(Y): 21.4633 dB
  -> New BEST RGB PSNR: 20.5771 dB at 

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x76d7fa4532e0><function _MultiProcessingDataLoaderIter.__del__ at 0x76d7fa4532e0><function _MultiProcessingDataLoaderIter.__del__ at 0x76d7fa4532e0><function _MultiProcessingDataLoaderIter.__del__ at 0x76d7fa4532e0>



Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
                self._shutdown_workers()self._shutdown_workers()self._shutdown_workers()

[ZHESTAR] Epoch [196/1000]  Train MSE: 0.006747  Val PSNR(RGB): 22.2704 dB  Val PSNR(Y): 22.9334 dB


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


[ZHESTAR] Epoch [197/1000]  Train MSE: 0.006676  Val PSNR(RGB): 22.2660 dB  Val PSNR(Y): 22.9129 dB
[ZHESTAR] Epoch [198/1000]  Train MSE: 0.006941  Val PSNR(RGB): 22.2652 dB  Val PSNR(Y): 22.9226 dB
[ZHESTAR] Epoch [199/1000]  Train MSE: 0.006800  Val PSNR(RGB): 22.3064 dB  Val PSNR(Y): 22.9751 dB
  -> New BEST RGB PSNR: 22.3064 dB at epoch 199
[ZHESTAR] Epoch [200/1000]  Train MSE: 0.006645  Val PSNR(RGB): 22.2650 dB  Val PSNR(Y): 22.9212 dB
[ZHESTAR] Epoch [201/1000]  Train MSE: 0.006826  Val PSNR(RGB): 22.2623 dB  Val PSNR(Y): 22.9351 dB
[ZHESTAR] Epoch [202/1000]  Train MSE: 0.006631  Val PSNR(RGB): 22.2930 dB  Val PSNR(Y): 22.9547 dB
[ZHESTAR] Epoch [203/1000]  Train MSE: 0.006619  Val PSNR(RGB): 22.2690 dB  Val PSNR(Y): 22.9300 dB
[ZHESTAR] Epoch [204/1000]  Train MSE: 0.006639  Val PSNR(RGB): 22.2971 dB  Val PSNR(Y): 22.9581 dB
[ZHESTAR] Epoch [205/1000]  Train MSE: 0.006710  Val PSNR(RGB): 22.3024 dB  Val PSNR(Y): 22.9832 dB
[ZHESTAR] Epoch [206/1000]  Train MSE: 0.006798  Val

In [26]:
import os
import torch

# --- choose which model you're resuming ---
model_choice = "zhestar"   # "vpeg" or "zhestar"

# must match what you used originally
channels   = 32
num_blocks = 4

ckpt_dir        = f"checkpoints_{model_choice}_nano"
last_ckpt_path  = os.path.join(ckpt_dir, "last_model.pt")
best_ckpt_path  = os.path.join(ckpt_dir, "best_model.pt")

print("Loading from:", last_ckpt_path)
last_ckpt = torch.load(last_ckpt_path, map_location=device)

# --- recreate SAME model architecture ---
if model_choice == "vpeg":
    model = VPEG_NanoRestorer(
        in_channels=3,
        out_channels=3,
        channels=channels,
        num_blocks=num_blocks,
    ).to(device)
else:
    model = ZHEstar_NanoRestorer(
        in_channels=3,
        out_channels=3,
        channels=channels,
        num_blocks=num_blocks,
    ).to(device)

model.load_state_dict(last_ckpt["model"])
print("Model weights loaded.")

# --- recreate optimizer & scheduler and restore state ---
# use the same optimizer type & LR you had before
initial_lr = 5e-4  # or whatever you originally used
optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
optimizer.load_state_dict(last_ckpt["optimizer"])

# recreate scheduler with dummy args and load its state
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=1000, eta_min=1e-6
)
scheduler.load_state_dict(last_ckpt["scheduler"])

start_epoch = last_ckpt["epoch"]
print("Resuming from epoch:", start_epoch)

# how many *extra* epochs you want
extra_epochs =500
target_epoch = start_epoch + extra_epochs
print("Training will continue up to epoch:", target_epoch)

# ---- load best PSNR so far (if best_ckpt exists) ----
if os.path.exists(best_ckpt_path):
    best_ckpt = torch.load(best_ckpt_path, map_location=device)
    best_val_psnr_rgb = best_ckpt.get("val_psnr_rgb", best_ckpt.get("val_psnr", 0.0))
    best_epoch        = best_ckpt["epoch"]
    print(f"Best so far: {best_val_psnr_rgb:.4f} dB at epoch {best_epoch}")
else:
    best_val_psnr_rgb = last_ckpt.get("val_psnr_rgb", last_ckpt.get("val_psnr", 0.0))
    best_epoch        = start_epoch
    print(f"No best_ckpt found, starting best PSNR from last_ckpt: {best_val_psnr_rgb:.4f} dB")


Loading from: checkpoints_zhestar_nano/last_model.pt
Model weights loaded.
Resuming from epoch: 2500
Training will continue up to epoch: 3000
Best so far: 23.1078 dB at epoch 2459


In [27]:
for epoch in range(start_epoch + 1, target_epoch + 1):
    # ---------- TRAIN ----------
    model.train()
    running_loss = 0.0

    for lr_img, hr_img in train_loader:
        lr_img = lr_img.to(device, non_blocking=True)
        hr_img = hr_img.to(device, non_blocking=True)

        optimizer.zero_grad()
        sr = model(lr_img)
        loss = criterion(sr, hr_img)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * lr_img.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # ---------- VALID ----------
    model.eval()
    val_psnr_rgb_sum = 0.0
    val_psnr_y_sum   = 0.0

    with torch.no_grad():
        for lr_img, hr_img in val_loader:
            lr_img = lr_img.to(device, non_blocking=True)
            hr_img = hr_img.to(device, non_blocking=True)

            sr = model(lr_img)
            val_psnr_rgb_sum += calc_psnr_rgb(sr, hr_img)
            val_psnr_y_sum   += calc_psnr_y(sr, hr_img)

    val_psnr_rgb = val_psnr_rgb_sum / len(val_loader)
    val_psnr_y   = val_psnr_y_sum / len(val_loader)

    scheduler.step()

    print(f"[{model_choice.upper()} RESUME] Epoch [{epoch}/{target_epoch}]  "
          f"Train MSE: {train_loss:.6f}  "
          f"Val PSNR(RGB): {val_psnr_rgb:.4f} dB  "
          f"Val PSNR(Y): {val_psnr_y:.4f} dB")

    # ---------- SAVE LAST ----------
    torch.save(
        {
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "val_psnr_rgb": val_psnr_rgb,
            "val_psnr_y": val_psnr_y,
        },
        last_ckpt_path,
    )

    # ---------- SAVE BEST ----------
    if val_psnr_rgb > best_val_psnr_rgb:
        best_val_psnr_rgb = val_psnr_rgb
        best_epoch        = epoch
        torch.save(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "val_psnr_rgb": val_psnr_rgb,
                "val_psnr_y": val_psnr_y,
            },
            best_ckpt_path,
        )
        print(f"  -> New BEST RGB PSNR: {best_val_psnr_rgb:.4f} dB at epoch {best_epoch}")


[ZHESTAR RESUME] Epoch [2501/3000]  Train MSE: 0.005612  Val PSNR(RGB): 23.0849 dB  Val PSNR(Y): 23.6962 dB
[ZHESTAR RESUME] Epoch [2502/3000]  Train MSE: 0.005742  Val PSNR(RGB): 23.0889 dB  Val PSNR(Y): 23.6934 dB
[ZHESTAR RESUME] Epoch [2503/3000]  Train MSE: 0.005779  Val PSNR(RGB): 23.0864 dB  Val PSNR(Y): 23.6984 dB
[ZHESTAR RESUME] Epoch [2504/3000]  Train MSE: 0.005605  Val PSNR(RGB): 23.0952 dB  Val PSNR(Y): 23.6985 dB
[ZHESTAR RESUME] Epoch [2505/3000]  Train MSE: 0.005845  Val PSNR(RGB): 23.0787 dB  Val PSNR(Y): 23.6747 dB
[ZHESTAR RESUME] Epoch [2506/3000]  Train MSE: 0.005780  Val PSNR(RGB): 23.0797 dB  Val PSNR(Y): 23.6796 dB
[ZHESTAR RESUME] Epoch [2507/3000]  Train MSE: 0.005471  Val PSNR(RGB): 23.1058 dB  Val PSNR(Y): 23.7101 dB
[ZHESTAR RESUME] Epoch [2508/3000]  Train MSE: 0.005755  Val PSNR(RGB): 23.0913 dB  Val PSNR(Y): 23.6942 dB
[ZHESTAR RESUME] Epoch [2509/3000]  Train MSE: 0.005724  Val PSNR(RGB): 23.0856 dB  Val PSNR(Y): 23.6863 dB
[ZHESTAR RESUME] Epoch [2510

In [29]:
import torch
import torch.onnx

ckpt_dir = f"checkpoints_{model_choice}_nano"
best_ckpt_path = os.path.join(ckpt_dir, "best_model.pt")

checkpoint = torch.load(best_ckpt_path, map_location="cpu")

channels   = 32   # must match what you used for training
num_blocks = 4

# Recreate the SAME model architecture as training
if model_choice == "vpeg":
    export_model = VPEG_NanoRestorer(
        in_channels=3,
        out_channels=3,
        channels=channels,
        num_blocks=num_blocks,
    )
else:  # "zhestar"
    export_model = ZHEstar_NanoRestorer(
        in_channels=3,
        out_channels=3,
        channels=channels,
        num_blocks=num_blocks,
    )

export_model.load_state_dict(checkpoint["model"])
export_model.eval()

dummy_input = torch.randn(1, 3, 256, 256)

onnx_path = f"{model_choice}_nano_1x3x256x256.onnx"

torch.onnx.export(
    export_model,
    dummy_input,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    opset_version=17,         # adjust if your tool needs other
    do_constant_folding=True,
    dynamic_axes=None         # fixed shape for clean NPU conversion
)

print("Exported ONNX to:", onnx_path)


Exported ONNX to: zhestar_nano_1x3x256x256.onnx


In [15]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
